In [5]:

import folium
import pandas as pd

In [6]:
utah_df= pd.read_csv("1871-utah-postmaster-salaries.csv")
print(utah_df.sample(5))


utah_df.dtypes

       PO_Name  County State  PM_Salary   Latitude   Longitude
119     Summit    Iron    UT         40  37.801087 -112.936335
81      Newton   Cache    UT         12  41.862705 -111.990784
84       Ophir  Tooele    UT         12  40.369390 -112.258557
75        Mona    Juab    UT         18  39.816621 -111.850213
73   Millville   Cache    UT         22  41.680765 -111.824109


PO_Name       object
County        object
State         object
PM_Salary      int64
Latitude     float64
Longitude    float64
dtype: object

In [7]:
utah_map_empty = folium.Map(location=[40, -111], zoom_start=6)
utah_map_empty


## Creating a Reusable Function for Maps

In [19]:
def create_empty_map():
    return folium.Map(location=[40, -111], zoom_start=6) ## this is utah map change location if copying code

utah_map = create_empty_map()
utah_map



In [10]:
# Check for columns with missing values
missing_values = utah_df.isna().sum()
print(missing_values)


PO_Name      0
County       0
State        0
PM_Salary    0
Latitude     4
Longitude    4
dtype: int64


In [11]:
utah_df_locations = utah_df[utah_df['Latitude'].notna()]

print(len(utah_df))
print(len(utah_df_locations))

136
132


In [12]:
folium.Marker(location=[38.41, -112.339], popup="Adamsville Post Office").add_to(utah_map)
utah_map


In [13]:
# Melanie Walsh function we will adadpt to our dataset:
# def create_map_markers(row, map_name):
#    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

def create_map_markers(row, map_name):
    folium.Marker(location=[row['Latitude'], row['Longitude']], 
                  popup=f"{row['Name']}: number of ppl working there: {row['People']}",
                  tooltip=f"Dept: {row['Department']}" 
                 ).add_to(map_name)
#that show the name of the location, the government agency, and the number of people working there

In [21]:
#create a base empty map
utah_map = create_empty_map()

#generate a random row of data
sample_row = utah_df_locations.sample(1)

#use our function on the random row
create_map_markers(sample_row, utah_map)

#display the map
utah_map


KeyError: 'Name'

In [ ]:
# Method 1: Using a for loop to iterate through our dataframe and add markers sequentially
# initialize an empty map
utah_map = create_empty_map()

# iterrows() allows you to loop through a dataframe row by row and return the index position + the row
for index, row in utah_df_locations.iterrows():
    print(f"Name of post office:", row[0])

#now let's iterate through and call our function for each row
for index, row in utah_df_locations.iterrows():
    create_map_markers(row, utah_map)

utah_map


In [ ]:
# Method 2: Using .apply() to add markers with our function for all rows
# initialize an empty map
utah_map = utah_map_empty

# Now apply this function to each row in our filtered DataFrame
# For each row, we'll pass:
# 1. The row itself (handled automatically by .apply())
# 2. Our map object (we need to specify this explicitly)
# 3. The "axis" value for .apply() to indicate we want to process row by row
# .apply() allows you to apply a function to each row in the dataframe
utah_df_locations.apply(
    create_map_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [23]:
# Melanie Walsh function we will edit:
#def create_ICE_map_markers(row, map_name):
#    folium.CircleMarker(location=[row['lat'], row['lon']], raidus=100, fill=True,
#                       popup=folium.Popup(f"{row['Name'].title()} <br> {row['City'].title()}, {row['State']}", max_width=200),
#                       tooltip=f"{row['Name'].title()} <br> {row['City'].title()}, {row['State']}"
#                       ).add_to(map_name)

def create_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=1,
                       fill=True,
                       popup=folium.Popup(f"{row['PO_Name'].title()}", max_width=200),
                       tooltip=f"{row['PO_Name'].title()}"
                       ).add_to(map_name)


In [25]:
# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(create_circle_markers, map_name=utah_map, axis="columns")

utah_map


In [27]:
# alter map appearance
def create_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=3,
                       color='lavender',
                       fill=True,
                       fill_color='red',
                       fill_opacity=0.6,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [ ]:
# make new function to create circle markers sized by postmaster salary
def create_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['PM_Salary'],
                       fill=True,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_sized_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [29]:
# make new function to create circle markers sized by postmaster salary - this time adjusting the radius size in pixels to make it more legible
def create_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['PM_Salary']/100,
                       fill=True,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_sized_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [ ]:
utah_df_locations.describe()

In [31]:
def add_salary_buckets(salary):
    # Create a new column for the salary bucket
    if salary < 50:
        bucket = 'Low Salary'
    elif salary >= 50 and salary < 250:
        bucket = 'Medium Salary'
    elif salary >= 250 and salary < 1000:
        bucket = 'High Salary'
    else:
        bucket = 'Very High Salary'
    return bucket


In [ ]:
#test out the function
add_salary_buckets(2000)


In [ ]:
utah_df_locations['Salary_Bucket'] = utah_df_locations['PM_Salary'].apply(add_salary_buckets)
utah_df_locations.head()


In [ ]:
# create a function to add marker sizes based on the salary bucket
##def add_marker_sizes(category):
    if category == 'Low Salary':
        return 4
    elif category == 'Medium Salary':
        return 8
    elif category == 'High Salary':
        return 12
    else:
        return 16

#test out the function



In [33]:
utah_df_locations['Marker_Size'] = utah_df_locations['Salary_Bucket'].apply(add_marker_sizes)
utah_df_locations.head(10)


KeyError: 'Salary_Bucket'

In [ ]:
# make new function to create circle markers sized by salary category
def create_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['Marker_Size'],
                       fill=True,
                       opacity=0.6,
                       popup=folium.Popup(f"Post Office: {row['PO_Name'].title()}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}"
                       ).add_to(map_name)

# initialize an empty map
utah_map = create_empty_map()

# call our function for each row
utah_df_locations.apply(
    create_sized_circle_markers, # The function to apply
    map_name=utah_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

utah_map


In [35]:
employee_df = pd.read_csv("1877-official-register.csv")

In [37]:
employee_df.sample(4)

,Name,State,Department,Type,People,Latitude,Longitude
438,Yankton,SD,Judicial,Court,5,42.871109,-97.397281
826,Fort Larned,KS,War Department,Outpost,36,38.183056,-99.218056
577,Salem,OR,Treasury Department,Internal Revenue_Deputy Collector,1,44.942898,-123.035096
674,Ano Nuevo,CA,Treasury Department,Lighthouse,2,37.133056,-122.333056


In [39]:
missing_values= employee_df.isna().sum()
print(missing_values)


Name          0
State         0
Department    0
Type          0
People        0
Latitude      0
Longitude     0
dtype: int64


In [60]:
us_map = create_empty_map()
employee_df.sample(10).apply(create_map_markers,
                 map_name=us_map,
                 axis='columns'
                 )

us_map



In [71]:
employee_df.describe()

,People,Latitude,Longitude
count,903.000000,903.000000,903.000000
mean,19.176080,38.974548,-108.959857
std,66.293517,5.508786,10.516498
min,1.000000,25.898333,-124.736600
25%,1.000000,35.584746,-120.409982
50%,1.000000,39.150171,-107.779780
75%,4.000000,42.871109,-97.743061
max,881.000000,48.966377,-93.322350


In [83]:
def add_people_buckets(people):
    if people <5:
        bucket = "Small Office"
    elif people >=5 and people <20:
        bucket = "Medium Office"
    elif people >=20 and people <100:
        bucket = "Large Office"
    else:
        bucket = "Huge Office"
    return bucket



add_people_buckets(30)

'Large Office'

In [87]:
employee_df['Office Size']= employee_df['People'].apply(add_people_buckets)

employee_df.head()

,Name,State,Department,Type,People,Latitude,Longitude,Office Size
0,Prescott,AZ,Department of the Interior,General Land Office Receivers,1,34.540024,-112.468503,Small Office
1,Florence,AZ,Department of the Interior,General Land Office Receivers,1,33.031451,-111.387343,Small Office
2,San Francisco,CA,Department of the Interior,General Land Office Receivers,1,37.774929,-122.419416,Small Office
3,Marysville,CA,Department of the Interior,General Land Office Receivers,1,39.145725,-121.591355,Small Office
4,Humboldt,CA,Department of the Interior,General Land Office Receivers,1,40.745005,-123.869509,Small Office


In [97]:
#create markers based on office size

def add_marker_sizes(category):
    if category == 'Small Office':
        return 4
    elif category == 'Medium Office':
        return 8
    elif category == 'Large Office':
        return 12
    else:
        return 16


employee_df['Marker_Size'] = employee_df['Office Size'].apply(add_marker_sizes)  

employee_df.sample(3)

,Name,State,Department,Type,People,Latitude,Longitude,Office Size,Marker Size,Marker_Size
588,Marshall,TX,Treasury Department,Internal Revenue_Deputy Collector,1,32.544871,-94.367418,Small Office,4,4
345,Albany,OR,Judicial,Commissioner,1,44.636511,-123.105928,Small Office,4,4
311,Juniata,NE,Judicial,Commissioner,1,40.590014,-98.506175,Small Office,4,4


In [51]:
# make new function to create circle markers sized by salary category
def create_new_sized_circle_markers(row, map_name):
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                       radius=row['Marker_Size'],
                       fill=True,
                       opacity=0.6,
                       popup=folium.Popup(f"{row['Name']}: number of ppl working there: {row['People']}", max_width=200),
                       tooltip=f"Postmaster Salary: ${row['PM_Salary']}" #change dis
                       ).add_to(map_name)

# initialize an empty map
us_map = create_empty_map()

# call our function for each row
employee_df.apply(
    create_sized_circle_markers, # The function to apply
    map_name=us_map, # Additional argument to pass to the function
    axis='columns' # Process row by row instead of column by column
)

us_map


KeyError: 'PM_Salary'